In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [4]:
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name='div', attrs={'class':'row'}):
        company = div.find_all(name='span', attrs={'class':'company'})
    if len(company) > 0:
        for b in company:
            companies.append(b.text.strip())
    else:
        sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
        for span in sec_try:
            companies.append(span.text.strip())
    return(companies)
 
# extract_company_from_result(soup)

In [5]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)
# extract_location_from_result(soup)

In [6]:
def extract_salary_from_result(soup): 
    salaries = []
    for div in soup.find_all(name='div', attrs={'class':'row'}):
        try:
            salaries.append(div.find('nobr').text)
        except:
            try:
                div_two = div.find(name='div', attrs={'class':'sjcl'})
                div_three = div_two.find('div')
                salaries.append(div_three.text.strip())
            except:
                salaries.append('Nothing_found')
    return(salaries)

# extract_salary_from_result(soup)

In [7]:
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll('span', attrs={'class': 'summary'})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)

# extract_summary_from_result(soup)

In [3]:
max_results_per_city = 3000 # Not necessary to put a very high value, 3000 is more than enough

city_set = ['Paris', 'Marseille', 'Toulouse', 'Lyon', 'Bordeaux', 'Nice', 'Strasbourg', 'Lille', 'Nantes','Nancy'] #Cities you want to scrap for offers

cols = ['city', 'location', 'job_title', 'company_name', 'summary', 'date', 'salary'] #Columns creation for future dataframe

In [4]:
# assembling the entire webscraping process
job_post_title = []
job_post_company = []
job_post_location = []
job_post_summary = []
job_post_date = []
job_post_salary = []
job_post_city = []

for city in city_set:
    
    
    for start in range(0, max_results_per_city, 10):

        page = requests.get('http://www.indeed.fr/emplois?q=data&l=' + str(city) + '&start=' + str(start))
        time.sleep(1)
        soup = BeautifulSoup(page.text, "lxml")
        

        for div in soup.find_all(name="div", attrs={"class":"row"}):
            
            
            job_post_city.append(city)
            
            #LOCATION
  
            location = div.find(name='span', attrs={'class': 'location accessible-contrast-color-location'})

            if location:
                job_post_location.append(location.text)
            else:
                job_post_location.append("NAN")
            
            #TITLE

            title = div.find(name="a", attrs={"data-tn-element":"jobTitle"})

            if title == None :
                job_post_title.append("Nan")
            else :
                job_post_title.append(title.text)

            #COMPANY

            company = div.find(name="span", attrs={"class":"company"})

            if company == None :
                job_post_company.append("Nan")
            else :
                job_post_company.append(company.text.strip())


            #SUMMARY

            summary = div.find(name="div", attrs={"class":"summary"})

            if summary == None :
                job_post_summary.append("Nan")
            else :
                job_post_summary.append(summary.text)

            #DATE

            date = div.find(name="span", attrs={"class":"date"})

            if date == None :
                job_post_date.append("Nan")
            else :
                job_post_date.append(date.text)

            #SALARY

            salary = div.find(name="span", attrs={"class":"salaryText"})

            if salary == None :
                job_post_salary.append("Nan")
            else :
                job_post_salary.append(salary.text)

In [5]:
# Assembling dataframe
df = pd.DataFrame()    
df['city'] = job_post_city
df['location'] = job_post_location
df['title'] = job_post_title
df['company_name'] = job_post_company
df['summary'] = job_post_summary
df['date'] = job_post_date
df['salary'] = job_post_salary

In [6]:
df.shape

(54798, 7)

In [7]:
data = df.copy()

In [8]:
data.to_csv('DATA_offer_city.csv', index=False)